In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
from math import exp, ceil, floor
from scipy.spatial import distance
from matplotlib import pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
iris = datasets.load_iris()

In [3]:
X = pd.DataFrame(iris.data)
y = pd.DataFrame(iris.target)

In [4]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [5]:
df = pd.DataFrame(iris.data)

In [6]:
df['y'] = pd.DataFrame(y)

In [7]:
df = df[(df['y'] == 1) | (df['y'] == 2)]

In [8]:
y = pd.DataFrame(df.y)
X = pd.DataFrame(df.drop('y', axis=1))

In [9]:
X.shape

(100, 4)

In [10]:
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))

In [11]:
X.shape

(100, 4)

In [13]:
X_train.shape

(80, 4)

In [14]:
y_train.shape

(80, 1)

In [15]:
data = pd.DataFrame(np.hstack([y_train, X_train]))

In [16]:
data.shape

(80, 5)

## Вероятностный взгляд. Бинарная классификация
Рассмотрим задачу классификации. Предположим, что ответ нашей модели $h_\theta(x_i)$ получен на основе логистической регрессии ($\sigma(Wx_i + b)$). Ее значения лежат в диапазоне от 0 до 1, что может быть интерпретировано, как вероятность, что $x_i$ принадлежит positive классу. Если вероятность $< 0.5$ - мы делаем предсказание о принадлежности отрицательному классу, а если $>= 0.5$ - положительному. Следовательно:

$$p(y_i = 1 \vert x_i)  = h_\theta(x_i)$$
$$p(y_i = 0 \vert x_i) = 1 - h_\theta(x_i)$$

Мы можем соединить это в одно уравнение:

$$p(y_i | x_i) = [h_\theta(x_i)]^{(y_i)} [1 - h_\theta(x_i)]^{(1 - y_i)}$$

Опять-таки из предположения, что наши данные независимы и одинаково распределены перейдем к правдоподобию:

$$L(x, y) = \prod_{i = 1}^{N}[h_\theta(x_i)]^{(y_i)} [1 - h_\theta(x_i)]^{(1 - y_i)}$$

Точно так же, как в случае MSE, возмьмем логарифм и инвертируем знак. Получим наш loss:

$$J = -\sum_{i=1}^{N} y_i\log (h_\theta(x_i)) + (1 - y_i)\log(1 - h_\theta(x_i))$$

In [17]:
weights = np.zeros(4)

In [18]:
features = np.array(X_train)

In [19]:
labels = np.array(y_train.values).reshape(80,)

In [20]:
lr = 0.01

In [21]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [22]:
def predict(features, weights):
    '''Возвращает массив массив вероятностей, что класс == 1'''
    z = np.dot(features, weights)
    return sigmoid(z)

In [23]:
def cost_function(features, labels, weights):
    '''Реализуем функцию потерь для бинарной классификации со средней абсолютной ошибкой'''
    observations = len(labels)
    predictions = predict(features, weights)
    #Take the error when label=1
    class1_cost = -labels*np.log(predictions)
    #Take the error when label=0
    class2_cost = (1-labels)*np.log(1-predictions)
    #Take the sum of both costs
    cost = class1_cost - class2_cost
    #Take the average cost
    cost = cost.sum()/observations
    return cost

In [24]:
def update_weights(features, labels, weights, lr):
    '''Реализуем алгоритм градиентного спуска'''
    N = len(features)
    #1 - Get Predictions
    predictions = predict(features, weights)
    #2 Transpose features from (200, 3) to (3, 200)
    # So we can multiply w the (200,1) cost matrix.
    # Returns a (3,1) matrix holding 3 partial derivatives --
    # one for each feature -- representing the aggregate
    # slope of the cost function across all observations
    gradient = np.dot(features.T, predictions - labels)
    #3 Take the average cost derivative for each feature
    gradient /= N
    #4 - Multiply the gradient by our learning rate
    gradient *= lr
    #5 - Subtract from our weights to minimize cost
    weights -= gradient
    return weights

In [25]:
def decision_boundary(prob):
    return 1 if prob >= .5 else 0

In [26]:
def classify(preds):
    #decision_boundary = np.vectorize(decision_boundary)
    return decision_boundary(preds).flatten()

In [27]:
def train(features, labels, weights, lr, iters):
    cost_history = []
    for i in range(iters):
        weightsold = weights
        weights = update_weights(features, labels, weights, lr)
        #Calculate error for auditing purposes
        cost = cost_function(features, labels, weights)
        cost_history.append(cost)
        # Log Progress
        if i % 1000 == 0:
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
        if cost<10**-10:
            print ("Алгоритм сошелся на шаге "+ str(i))
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
            break
    return weights, cost_history

In [28]:
W, C = train(features, labels, weights, 0.01, 100000)

iter: 0 cost: 0.6884063930620666
[0.00285552 0.00152756 0.00440659 0.00420512]
iter: 1000 cost: 0.1571661907314658
[ 0.2975576  -0.13422338  1.35471634  1.32351586]
iter: 2000 cost: 0.09427132981503857
[ 0.1753828  -0.35430905  1.92112384  1.80751898]
iter: 3000 cost: 0.06287637757598512
[ 0.05767191 -0.48645441  2.34377886  2.12701728]
iter: 4000 cost: 0.042602234883608134
[-0.04693184 -0.56759438  2.69771733  2.37064004]
iter: 5000 cost: 0.027663167611299055
[-0.14081719 -0.61775735  3.01052958  2.57057228]
iter: 6000 cost: 0.015746033626563037
[-0.22652764 -0.64790535  3.2957675   2.7421702 ]
iter: 7000 cost: 0.005733099302533717
[-0.30597054 -0.66449828  3.56118855  2.89395209]
Алгоритм сошелся на шаге 7644
iter: 7644 cost: -4.4714437737347625e-06
[-0.35447553 -0.66996865  3.72398526  2.98372524]


In [29]:
#Проверим адекватность метода на стандартном алгоритме "из коробки"

In [30]:
logreg = LogisticRegression()

In [31]:
logreg.fit(X, y)

/home/pavel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
logreg.coef_

array([[-0.28109636, -0.59297763,  2.21267607,  2.38820582]])

In [33]:
print(W)

[-0.35447553 -0.66996865  3.72398526  2.98372524]


In [34]:
logreg.intercept_

array([0.08787821])

In [35]:
y_pred = logreg.predict(X_test)

In [36]:
y_pred

array([1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1])

In [37]:
accuracy_score(y_test, y_pred)

0.95

In [38]:
pred = predict(features, W)

In [39]:
#Реализуем функцию прогнозирования принадлежности к классу
def func(W, X):
    klass = []
    for i in X.values:
        sg = (1/(1+exp(W[0]*i[0]+W[1]*i[1]+W[2]*i[2]+W[3]*i[3])))
        if sg < 0.5:
            a = 2
        else: 
            a = 1
        klass.append(a)
    return klass

In [40]:
y_pred = func(W, X_test)

In [41]:
y_pred

[1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1]

In [42]:
accuracy_score(y_test, y_pred)

1.0

In [43]:
# Реализуем функцию обновления весов методом Несторов Моментум

def update_weights_nesterov_momentum(features, labels, weights, lr, gamma):
    '''Реализуем алгоритм градиентного спуска'''
    N = len(features)
    momentumold = 0
    momentum = 0
    #1 - Получаем предсказания
    predictions = predict(features, weights)
    #2 Транспонируем матрицу факторов, так что мы сможем умножить 
    # So we can multiply w the (200,1) cost matrix.
    # Returns a (3,1) matrix holding 3 partial derivatives --
    # one for each feature -- representing the aggregate
    # slope of the cost function across all observations
    gradient = np.dot(features.T, predictions - labels)
    #3 Take the average cost derivative for each feature
    gradient /= N
    momentumold = momentum
    momentum = momentumold*gamma+ (1-gamma)*gradient
    #4 - Multiply the gradient by our learning rate
    gradient = gradient*lr+momentum
    #5 - Subtract from our weights to minimize cost
    weights -= gradient
    return weights

In [44]:
def train_nesterov_momentum(features, labels, weights, lr, iters, gamma):
    cost_history = []
    for i in range(iters):
        weightsold = weights
        weights = update_weights_nesterov_momentum(features, labels, weights, lr, gamma)
        #Calculate error for auditing purposes
        cost = cost_function(features, labels, weights)
        cost_history.append(cost)
        # Log Progress
        if i % 1000 == 0:
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
        if cost<10**-10 or cost == 0:
            print ("Алгоритм сошелся на шаге "+ str(i))
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
            break
    return weights, cost_history

In [45]:
weights = np.zeros(4)

In [46]:
W, C = train_nesterov_momentum(features, labels, weights, 0.01, 10000, 0.9)

iter: 0 cost: 0.6429820966378591
[0.03141073 0.01680318 0.04847249 0.04625634]
Алгоритм сошелся на шаге 693
iter: 693 cost: -1.5961572965350611e-06
[-0.35392491 -0.67049307  3.72362122  2.98445991]


In [47]:
pred = predict(features, W)

In [48]:
y_pred = func(W, X_test)

In [49]:
y_pred

[1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1]

In [50]:
accuracy_score(y_test, y_pred)

1.0

In [51]:
# реализуем обновление весов методом RMSProp

In [52]:
def update_weights_RMSProp(features, labels, weights, lr, gamma, epsilon):
    '''Реализуем алгоритм градиентного спуска'''
    N = len(features)
    momentumold = 0
    momentum = 0

    predictions = predict(features, weights)

    gradient = np.dot(features.T, predictions - labels)

    gradient /= N
    momentumold = momentum
    momentum = momentumold*gamma+ (1-gamma)*gradient**2
    weights = weights - (lr/(np.sqrt(momentum+epsilon))*gradient)
    return weights

In [53]:
def train_RMSProp(features, labels, weights, lr, iters, gamma, epsilon):
    cost_history = []
    for i in range(iters):
        weightsold = weights
        weights = update_weights_RMSProp(features, labels, weights, lr, gamma, epsilon)
        #Calculate error for auditing purposes
        cost = cost_function(features, labels, weights)
        cost_history.append(cost)
        # Log Progress
        if i % 1000 == 0:
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
        if cost<10**-10 or cost == 0:
            print ("Алгоритм сошелся на шаге "+ str(i))
            print ("iter: "+str(i) + " cost: "+str(cost))
            print (weights)
            break
    return weights, cost_history

In [54]:
weights = np.zeros(4)

In [55]:
W, C = train_RMSProp(features, labels, weights, 0.01, 10000, 0.9, 10**-4)

iter: 0 cost: 0.6538597348551767
[0.03143063 0.03096621 0.03154166 0.03153374]
Алгоритм сошелся на шаге 141
iter: 141 cost: -0.0004917324427380354
[-0.34909885 -0.73513552  3.64428201  3.17744491]


In [56]:
pred = predict(features, W)

In [57]:
y_pred = func(W, X_test)

In [58]:
y_pred

[1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1]

In [59]:
accuracy_score(y_test, y_pred)

1.0

In [60]:
# Итого: метод Нестерова в 10 раз быстрее обычного градиентного спуска, 
#метод RMSProp - в 7 раз быстрее метода Нестерова

# Далее просьба не проверять - черновые варианты реализации

In [61]:
# Реализуем алгоритм подбора весов логистической регрессии методом градиентного спуска
def GradientDescentLogReg(C, data, k):
    print('C=%d' %C)
    errorAccuracy=0
    l, m = data.shape
    weights=np.ones(m-1)
    distance_euclidean=0

    weightsDelta=np.ones(m-1)
    for step in range(10000):
        oldweightsDelta=weightsDelta
        weightsDelta=np.ones(m-1)

        for obj in data.values:
            y=obj[0]
            gradient=y*(1-1/(1+exp(-y*(weights[0]*obj[1]+weights[1]*obj[2]+weights[2]*obj[3]+weights[3]*obj[4]))))
            weightsDelta=list(map(lambda w,wd,x: wd+x*gradient-k*C*w,weights,weightsDelta,obj[1:]))

        #проверим Евклидово расстояние между соседними шагами, если менее порога, выходим из цикла
        distance_euclidean=distance.euclidean(weightsDelta,oldweightsDelta)
        if distance_euclidean<errorAccuracy:
            print('Дошли до заданной ошибки точности на шаге: %d' % step)
            break

        weights=list(map(lambda w,wd: w+wd*k/l,weights,weightsDelta))

    print('Евклидово расстояние=%.10f' %distance_euclidean)
    return weights

In [62]:
W = GradientDescentLogReg(1, data, 0.1)

C=1
Евклидово расстояние=0.0000000000


In [63]:
W

[-0.05273912057319143,
 -0.03672944110749239,
 0.24227699565417146,
 0.1845187338670464]